In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from collections import Counter


In [2]:
def fresh_data():
  df=pd.read_csv("log_dataset.csv")
  return df

In [3]:
df= fresh_data()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6708 entries, 0 to 6707
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   log     6708 non-null   object
 1   class   6708 non-null   object
dtypes: object(2)
memory usage: 104.9+ KB


In [4]:
df["class"].value_counts()

class
Benign                                  3707
Brute Force Attack                       361
Privilege Escalation                     287
Cross-Site Request Forgery (CSRF)        275
Malware and Ransomware                   249
Man-in-the-Middle (MITM)                 247
Cross-Site Scripting (XSS)               235
SQL Injection                            226
Distributed Denial of Service (DDoS)     211
Phishing and Credential Harvesting       200
Data Exfiltration                        169
Insider Threats                          136
Zero-Day Exploit                         120
Remote Code Execution (RCE)              119
Directory Traversal                      112
Phishing via Malicious Links              54
Name: count, dtype: int64

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Initialize the vectorizer
vectorizer = TfidfVectorizer(stop_words="english", max_features=1000)

# Fit the vectorizer on the log data and transform the logs into TF-IDF features
X_tfidf = vectorizer.fit_transform(df["log"])


In [6]:
def Splitting_Data(x,y):
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
  return X_train, X_test, y_train, y_test

In [7]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, classification_report
list_model=[]
def ml_models(X_train, X_test, y_train, y_test, models):
    for model_name, model in models.items():
        print(f"Model: {model_name}")

        # Train the model
        model.fit(X_train, y_train)

        # Predictions on the test data
        predictions_test = model.predict(X_test)

        # Predictions on the train data
        predictions_train = model.predict(X_train)

        # Metrics for test data
        accuracy_test = accuracy_score(y_test, predictions_test)
        f1_test = f1_score(y_test, predictions_test, average='weighted')
        recall_test = recall_score(y_test, predictions_test, average='weighted')

        print(f"Test Accuracy: {accuracy_test:.2f}")
        print(f"Test F1 Score: {f1_test:.2f}")
        print(f"Test Recall: {recall_test:.2f}")
        print(f"Test Classification Report:\n{classification_report(y_test, predictions_test)}")

        # Metrics for train data
        accuracy_train = accuracy_score(y_train, predictions_train)
        f1_train = f1_score(y_train, predictions_train, average='weighted')
        recall_train = recall_score(y_train, predictions_train, average='weighted')

        print(f"Train Accuracy: {accuracy_train:.2f}")
        print(f"Train F1 Score: {f1_train:.2f}")
        print(f"Train Recall: {recall_train:.2f}")
        print(f"Train Classification Report:\n{classification_report(y_train, predictions_train)}")
        list_model.append(model)
        print("\n" + "="*50 + "\n")
        return list_model


# **Hybrid Technique for Imblanced Data : SMOTEENN**

In [ ]:
from imblearn.combine import SMOTEENN

# Initialize SMOTEENN
smote_enn = SMOTEENN(random_state=42)

# Fit SMOTEENN and resample the dataset
X_resampled, y_resampled = smote_enn.fit_resample(X_tfidf, df["class"])


# **Add noise**

In [ ]:
def add_label_noise(df, column, noise_ratio=0.1):
    """
    Adds noise to the labels by randomly flipping them.
    :param df: DataFrame containing the data
    :param column: Column containing the labels
    :param noise_ratio: Percentage of labels to modify
    :return: DataFrame with noisy labels
    """
    df_noisy = df.copy()
    n_samples = int(len(df) * noise_ratio)
    indices_to_flip = random.sample(range(len(df)), n_samples)
    unique_labels = df[column].unique()
    for idx in indices_to_flip:
        current_label = df_noisy.loc[idx, column]
        df_noisy.loc[idx, column] = random.choice([label for label in unique_labels if label != current_label])
    return df_noisy


In [ ]:
import random

df_noisy = add_label_noise(df, column="class", noise_ratio=0.1)


In [ ]:
x=ml_models(Splitting_Data(X_resampled, y_resampled)[0], Splitting_Data(X_resampled, y_resampled)[1], Splitting_Data(X_resampled, y_resampled)[2], Splitting_Data(X_resampled, y_resampled)[3],models={

    "Logistic Regression"         : LogisticRegression(max_iter=10000000),
    "K-Nearest Neighbors"          : KNeighborsClassifier(n_neighbors=5),
    "Support Vector Machine"    : SVC(),
    "Decision Tree"             : DecisionTreeClassifier(max_depth=10, min_samples_split=10),
    "Random Forest"              : RandomForestClassifier(max_depth=10, n_estimators=100),
    "Gradient Boosting"            : GradientBoostingClassifier(),
    "Neural Network"              : MLPClassifier(max_iter=10000, hidden_layer_sizes=(50, 50), learning_rate_init=0.01)
  }
)

# **export Model:**

In [ ]:
import joblib

joblib.dump(x[0],'model_finale.pkl')
joblib.dump(vectorizer,"vectorizer.pkl")